In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-04-25 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-04-24 19:32:34,808 INFO: Initializing external client
2025-04-24 19:32:34,809 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:32:35,769 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 


In [7]:
df["pickup_hour"].max()

Timestamp('2025-03-06 04:00:00+0000', tz='Etc/UTC')

In [8]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,177,1.0,2025-03-03 15:00:00+00:00
1,72,2.0,2025-03-03 15:00:00+00:00
2,16,0.0,2025-03-03 15:00:00+00:00
3,209,4.0,2025-03-03 15:00:00+00:00
4,68,123.0,2025-03-03 15:00:00+00:00
...,...,...,...
10804,160,0.0,2025-03-06 04:00:00+00:00
10805,45,10.0,2025-03-06 04:00:00+00:00
10806,93,0.0,2025-03-06 04:00:00+00:00
10807,146,1.0,2025-03-06 04:00:00+00:00


In [9]:
#a.info()

In [10]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-04-24 19:32:39,743 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-24 19:32:39,750 INFO: Initializing external client
2025-04-24 19:32:39,750 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:32:40,521 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Current UTC time: 2025-04-24 23:32:39.743845+00:00
Next hour: 2025-04-25 00:00:00+00:00
Querying for date range: 2025-04-25 to 2025-04-26
Filtering for hour: 2025-04-25 00:00
Found 0 records


In [11]:
now = datetime.now(timezone.utc)

In [12]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10809 entries, 0 to 10808
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  10809 non-null  int32                  
 1   predicted_demand    10809 non-null  float64                
 2   pickup_hour         10809 non-null  datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 211.2 KB


In [14]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [15]:
dt = current_date.ceil('h')


In [16]:
# fs = get_feature_store()  
# fg = fs.get_feature_group(  
#     name=config.FEATURE_GROUP_MODEL_PREDICTION,  
#     version=1  
# )  
# query = fg.select_all() 
# # query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
# results = query.read()

In [17]:
# fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

In [18]:
# results

In [19]:
# def fetch_next_hour_predictions():
#     # Get current UTC time and round up to next hour
#     now = datetime.now(timezone.utc)
#     next_hour = (now + timedelta(hours=2)).replace(minute=0, second=0, microsecond=0)

#     fs = get_feature_store()
#     fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

#     # Then filter for next hour in the DataFrame
#     df_hour = df[df["pickup_hour"] == next_hour]

#     print(f"Current UTC time: {now}")
#     print(f"Next hour: {next_hour}")
#     print(f"Found {len(df_hour)} records")
#     return df_hour

In [20]:
# predictions = fetch_next_hour_predictions()

In [21]:
# fs = get_feature_store()
# fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



In [22]:
# df = fg.read()

In [23]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,177,1.0,2025-03-03 15:00:00+00:00
1,72,2.0,2025-03-03 15:00:00+00:00
2,16,0.0,2025-03-03 15:00:00+00:00
3,209,4.0,2025-03-03 15:00:00+00:00
4,68,123.0,2025-03-03 15:00:00+00:00
...,...,...,...
10804,160,0.0,2025-03-06 04:00:00+00:00
10805,45,10.0,2025-03-06 04:00:00+00:00
10806,93,0.0,2025-03-06 04:00:00+00:00
10807,146,1.0,2025-03-06 04:00:00+00:00


In [24]:
df["pickup_hour"].max()

Timestamp('2025-03-06 04:00:00+0000', tz='Etc/UTC')

In [25]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-04-25 00:00:00+00:00


In [26]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
